In [5]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()


In [3]:
start = '2015-04-15'
end = '2015-05-01'

In [7]:
engine = create_engine('sqlite:///../Base/hawaii.sqlite')

In [8]:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [16]:
Base.classes.keys()[0]

'measurement'

In [17]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [18]:
session = Session(engine)

In [20]:
for row in session.query(Station.name).limit(5).all():
    print(row)

('WAIKIKI 717.2, HI US',)
('KANEOHE 838.1, HI US',)
('KUALOA RANCH HEADQUARTERS 886.9, HI US',)
('PEARL CITY, HI US',)
('UPPER WAHIAWA 874.3, HI US',)
